<h1 align="center">Домен применимости (общая концепция, валидация, отбор, оценка)</h1>

Организацией Экономического Сотрудничества и Развития (ОЭСР) сформулирован набор принципов, которым должны отвечать модели «структура-свойство» для того, чтобы их можно было использовать для нормативных целей.
Модели "структура-свойства" должны удовлетворять следующим принципам:



**Общая идея определения области применимости моделей: в область применимости модели должны попадать только те химические объекты, которые похожи на объекты, использованные для ее построения.**

**Условно методы, используемые для определения областей применимости моделей "структура-свойства",  можно разделить на следующие:**

* основанные на интервалах значений дескрипторов:
 1. Bounding box
 2. PCA Bounding box
 3. TOPKAT Optimap Prediction Space
 4. Standartization approach
 5. Fragment Control
* основанные на расстоянии до модели в химическом пространстве дескрипторов:
 1. Leverage
 2. Nearest Neighbors approach
 3. Euclidean, Mahalanobis, City Block Distances
 4. Hotelling T2 Test
 5. DModX (Distance to the Model in X-Space)
 6. Tanimoto Similarity
* на основе оценки дисперсии плотности прогноза:
 1. Parametric method
 2. Nonparametric method (Gaussian Process)
* базирующиеся на оценке плотностей распределения в химическом пространстве дескрипторов
* на основе алгоритмов одноклассовой классификации:
 1. One-Class Support Vector Machine
 2. Isolation Forest
* прочие 
  
Несмотря на широкое разнообразие предложенных подходов к оценке домена применимости моделей, большинство их объединяет общая идея – в домен применимости модели должны попадать только те химические объекты, которые похожи на объекты, использованные для ее построения.

Все необходимые функции и объекты реализованы в пакетах scikit-learn (sklearn) [6] и CIMtools [7].

### Импортируем все необходимые библиотеки

In [1]:
from CIMtools.preprocessing import DictToConditions, ConditionsToDataFrame, \
                                   EquationTransformer, SolventVectorizer, Fragmentor, CGR
from CIMtools.applicability_domain import Box, ReactionTypeControl, Leverage
from CGRtools.files import RDFRead

# from os import environ
# from os.path import expanduser

from sklearn.model_selection import KFold, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

import numpy as np
import pandas as pd


### Функции для расчета характеристик качества моделей с использованием области применимости модели

Определим функции для расчета характеристик качества моделей с использованием домена применимости модели.

*Расчет среднеквадратичной ошибки (RMSE) для объектов внутри области применимости модели:*

In [2]:
def rmse_score_with_ad(Y_true, Y_pred, AD):
    if AD.sum():  # количество объектов внутри области применимости модели
        return np.sqrt(mean_squared_error(Y_true[AD], Y_pred[AD]))
    return np.inf

*Расчет коэффициента детерминации (R<sup>2</sup>) для объектов внутри области применимости модели:*

In [3]:
def r2_score_with_ad(Y_true, Y_pred, AD):
    if AD.sum():  # количество объектов внутри области применимости модели
        return r2_score(Y_true[AD], Y_pred[AD])
    return -np.inf

*Доля объектов, принадлежащих области применимости модели:*

In [4]:
def coverage(AD):
    return sum(AD)/len(AD)

### Генерируем фрагментные дескрипторы для реакций Дильс-Альдера 

Построение модели и оценка ее домена применимости будут показаны на наборе реакций Дильса-Альдера. Данный набор больше, чем использованный ранее, что позволит избежать ряда проблем небольших выборок, таких как смещенность и переобучение. Однако, все описанные манипуляции точно так же подходят и для моделирования свойств молекул.

In [5]:
data = RDFRead('data/DA.rdf').read()

В отличие от молекул, свойства реакций сильно зависят от условий проведения, таких как: температура и растворитель. Поэтому здесь мы подготовим конвейер для извлечения дескрипторов растворителей и температуры.

In [6]:
def extract_meta(x):
    return [y[0].meta for y in x]


features = ColumnTransformer([('temp', EquationTransformer('1/x'), ['temperature']),
                              ('solv', SolventVectorizer(), ['solvent.1']),
                              ('amount', 'passthrough', ['solvent_amount.1']),])

conditions = Pipeline([('meta', FunctionTransformer(extract_meta)),
                       ('cond', DictToConditions(solvents=('additive.1',), 
                                                 temperature='temperature', 
                                                 amounts=('amount.1',))),
                       ('desc', ConditionsToDataFrame()),
                       ('final', features)])

Конвейер выше принимает на вход объекты реакций и извлекает из них метаданные, в которых содержится информация об условиях. Температура преобразовывается в обратное значение (1/T, К<sup>-1</sup> ), а названия растворителей
заменяются на вектора из 13 физико-химических параметров.

In [7]:
graph = Pipeline([('CGR', CGR()),
                  ('frg', Fragmentor(max_length=4, useformalcharge=True)),
                  ('scaler', StandardScaler())])

Второй конвейер позволяет обработать структурную информацию реакций. Для этого воспользуемся подходом КГР (Конденсированного Графа Реакции). Поскольку фрагментные дескрипторы ISIDA поддерживают КГР, мы можем закодировать реакцию с вниманием на саму трансформацию (механизм реакции).

In [8]:
dsc = ColumnTransformer([('cond', conditions, [0]),
                         ('graph', graph, [0])])

Объединив оба конвейера, можно получить итоговый конвейер, который преобразует структуру и условия в дескрипторы. Интерфейс полученного объекта аналогичен интерфейсу описанных ранее генераторов дескрипторов.
Обратите внимание, что исходные данные из простого списка пришлось преобразовать во вложенные списки. Данное преобразование превратило список структур в матрицу с одной колонкой, потому что в scikit-learn свойства
объектов должны описываться в виде матриц.

In [9]:
X = dsc.fit_transform([[x] for x in data])

In [10]:
Y = np.array([float(r.meta['logK'])for r in data])  # предсказываемые свойства

### Делим данные (X, Y, data) на обучающую и тестовую выборки

In [11]:
X_train, X_test, \
y_train, y_test, \
data_train, data_test = train_test_split(X, Y, data, test_size=0.2, random_state=0)

In [12]:
X_train.shape, len(y_train), len(data_train)

((1492, 464), 1492, 1492)

In [13]:
X_test.shape, len(y_test), len(data_test)

((374, 464), 374, 374)

### Строим регрессионную модель

In [14]:
est = RandomForestRegressor(random_state=1, n_estimators=500, max_features=0.35, n_jobs=4)

In [15]:
est.fit(X_train, y_train)

RandomForestRegressor(max_features=0.35, n_estimators=500, n_jobs=4,
                      random_state=1)

### Предсказываем константу скорости (lоgK) реакций 

In [16]:
y_pred = est.predict(X_test)

Оцениваем предсказательную способность модели.

In [17]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [18]:
round(rmse, 2), round(r2, 2)

(np.float64(0.64), 0.89)

### Строим область применимости модели

### Bounding Box

В данном случае домен применимости моделей представляет собой D-мерный прямоугольный параллелепипед, определенный на основе максимальных и минимальных значений каждого дескриптора, используемого для построения модели. Объект принадлежит домену применимости моделей, если рассчитанные для него значения всех присутствующих в модели дескрипторов не выходят за границы соответствующего интервала. Метод не имеет внутренних настраиваемых параметров. Интерфейс метода аналогичен обычным моделям.

<img src='images/Bounding_Box.png'>

In [19]:
bb = Box()

bb.fit(X_train)  # обучаем метод

ad_bb = bb.predict(X_test)  # предсказываем принадлежность объектов из тестовой выборки 
                            # домену применимости модели

Проверяем, как изменилось качество модели с использованием Bounding Box в качестве области применимости модели.  
  
Для этого вызовем функции `rmse_score_with_ad( )`, `r2_score_with_ad( )` и рассчитаем показатели качества моделей для объектов внутри области применимости модели и для объектов вне области применимости модели.
  
После чего рассчитаем долю объектов из тестовой выборки, которые остались в области применимости модели после применения метода. Для этого вызовем функцию `coverage( )`.

In [20]:
rmse_inliers_with_bb = rmse_score_with_ad(y_test, y_pred, ad_bb)
r2_inliers_with_bb = r2_score_with_ad(y_test, y_pred, ad_bb)
coverage_inliers_bb = coverage(ad_bb)

In [21]:
rmse_outliers_with_bb = rmse_score_with_ad(y_test, y_pred, ~ad_bb)
r2_outliers_with_bb = r2_score_with_ad(y_test, y_pred, ~ad_bb)
coverage_outliers_bb = coverage(~ad_bb)  # ~ - инвертирование булевого значения

In [22]:
result_df = pd.DataFrame(
    {'RMSE': [f"{rmse_inliers_with_bb:.2f}", 
              f"{rmse_outliers_with_bb:.2f}"],  
     'R2': [f"{r2_inliers_with_bb:.2f}", 
            f"{r2_outliers_with_bb:.2f}"],
     'Coverage': [f"{coverage_inliers_bb:.3f}", 
                  f"{coverage_outliers_bb:.3f}"]}, 
    index=['Внутри', 'Снаружи'])

In [23]:
result_df

,RMSE,R2,Coverage
Внутри,0.59,0.90,0.987
Снаружи,2.16,0.42,0.013


Как видно из таблицы, реакции, не попавшие в домен применимости, предсказываются плохо.

## Reaction Type Control

Этот способ оценки домена применимости моделей предложен в нашей лаборатории [8] специально для моделирования связи «структура-реакционная способность» (Quantitative Structure-Reactivity Relationship, QSRR). Метод основан на использовании реакционных центров и их окружения. Тип реакционного центра определяется с помощью сигнатуры КГР, сгенерированной библиотекой **CGRtools**. Под реакционной сигнатурой понимается уникальное символьное представление
подструктуры реакции, обусловленное определенным окружением атомов реакционного центра. Создание сигнатуры включает в себя: 1 – кодирование реакции в виде КГР, 2 – выделение на КГР одного или нескольких реакционных центров (т.е. подграфов КГР, вершины которых связаны путями, проходящими исключительно по динамическим связям либо по смежным динамическим атомам, связанным обычными связями), выделение атомов окружения каждого из реакционных центров определенного радиуса (топологического расстояния от атома окружения до ближайшего динамического атома или атома, инцидентного динамической связи), 3 – введение канонической нумерации атомов реакционного центра с окружением с использованием алгоритма, аналогичного алгоритму Моргана, 4 – создание уникального алфавитно-цифрового (линейного) преставления. Радиус окружения, включенный в сигнатуру, являлся гиперпараметром метода. Если радиус равен 0, сигнатура реакции включает в себя только атомы реакционного центра. Обучение в данном методе представляет собой запоминание сигнатуры реакционных центров реакций. Если тестовая реакция с таким реакционный центром была в обучающей выборке, то объект принадлежит домену применимости модели.

In [24]:
rtc = ReactionTypeControl(env=2)

rtc.fit(data_train) # обучаем метод

ad_rtc = rtc.predict(data_test)

rmse_inliers_with_rtc = rmse_score_with_ad(y_test, y_pred, ad_rtc)
r2_inliers_with_rtc = r2_score_with_ad(y_test, y_pred, ad_rtc)
coverage_inliers_with_rtc = coverage(ad_rtc)

rmse_outliers_with_rtc = rmse_score_with_ad(y_test, y_pred, ~ad_rtc)
r2_outliers_with_rtc = r2_score_with_ad(y_test, y_pred, ~ad_rtc)
coverage_outliers_with_rtc = coverage(~ad_rtc)

In [25]:
result_df = pd.DataFrame(
    {'RMSE': [f"{rmse_inliers_with_rtc:.2f}", 
              f"{rmse_outliers_with_rtc:.2f}"],  
     'R2': [f"{r2_inliers_with_rtc:.2f}", 
            f"{r2_outliers_with_rtc:.2f}"],
     'Coverage': [f"{coverage_inliers_with_rtc:.3f}", 
                  f"{coverage_outliers_with_rtc:.3f}"]}, 
    index=['Внутри', 'Снаружи'])

In [26]:
result_df

,RMSE,R2,Coverage
Внутри,0.46,0.94,0.936
Снаружи,1.81,0.58,0.064


Данный тип домена применимости тоже отделил часть реакций за область применимости.

## Leverage

Для определения домена применимости модели в данном случае используется числовая характеристика (показатель
влиятельности), характеризующая расстояние между рассматриваемым химическим объектом и геометрическим центром облака точек, образованного входящими в обучающую выборку объектами. Показатель влиятельности (англ. *Leverage*) – это величина h, определяемая как  
*h = х<sup>T</sup>(X<sup>T</sup>X)<sup>-1</sup>х,*  
где *X* – матрица, первый столбец которой состоит из единиц, а элемент на пересечении *i*-ой строки и *(j+1)*-ого столбца равен значению *j*-ого дескриптора для *i*-ого химического объекта из обучающей выборки. Считается, что объект принадлежит домену применимости модели, если для него значение показателя влиятельности не превышает пороговое. Недостатком данного метода является отсутствие строгих правил в литературе по определению порогового значения. Однако мы можем найти оптимальный порог *h** в ходе процедуры внутренней перекрестной проверки с использованием некоторой метрики качества.

<img src='images/Leverage.png'>

In [27]:
lev = Leverage(threshold='auto') # если threshold='auto', то отсечка будет определяться как h*=3*(M+1)/N
                                 # если threshold='cv', то отсечка будет найдена с помощью внутренней 
                                 # кросс-валидации

lev.fit(X_train)

ad_lev = lev.predict(X_test)

rmse_inliers_with_lev = rmse_score_with_ad(y_test, y_pred, ad_lev)
r2_inliers_with_lev = r2_score_with_ad(y_test, y_pred, ad_lev)
coverage_inliers_with_lev = coverage(ad_lev)

rmse_outliers_with_lev = rmse_score_with_ad(y_test, y_pred, ~ad_lev)
r2_outliers_with_lev = r2_score_with_ad(y_test, y_pred, ~ad_lev)
coverage_outliers_with_lev = coverage(~ad_lev)

In [28]:
result_df = pd.DataFrame(
    {'RMSE': [f"{rmse_inliers_with_lev:.2f}", 
              f"{rmse_outliers_with_lev:.2f}"],  
     'R2': [f"{r2_inliers_with_lev:.2f}", 
            f"{r2_outliers_with_lev:.2f}"],
     'Coverage': [f"{coverage_inliers_with_lev:.3f}", 
                  f"{coverage_outliers_with_lev:.3f}"]}, 
    index=['Внутри', 'Снаружи'])

In [29]:
result_df

,RMSE,R2,Coverage
Внутри,0.51,0.92,0.960
Снаружи,1.95,0.49,0.040


Данный метод тоже показал аналогичную тенденцию.

Как видно из приведенных примеров, домен применимости позволяет контролировать, когда стоит доверять модели, а когда нет. Применительно к разработке лекарств, это позволит значительно сократить число ложноположительных или ложноотрицательных предсказаний модели. Особенно это полезно на стадии поиска структур с желаемой активностью.

C некоторыми другими методами определения области применимости моделей, а также с примерами использования домена применимости модели в процессе QSPR моделирования, можно ознакомиться по ссылке https://cimtools.readthedocs.io/en/latest/tutorial/applicability_domain.html#  